<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/ordinal_outcomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
# Fragility Metrics Toolkit: Ordinal Outcomes (Wilcoxon-Mann-Whitney / Proportional Odds)
# 25-NOV-2025
# Fully aligned with FRAGILITY_METRICS.md v10.1 §3.10 OFQ and §4.8 ORQ
#
# Single mode: gOR (generalized odds ratio) + 95% CI
#
# Output: p (approximate), fr (OFQ), nb (ORQ)
#
# Formulas (exact v10.1):
# - OFQ = |z_WMW - 1.96| / (1 + |z_WMW - 1.96|)
# - ORQ = |ln(gOR)| / (1 + |ln(gOR)|)
#
# Common applications:
# - Modified Rankin Scale (mRS) shift analysis
# - NIHSS ordinal analysis
# - Pain scale comparisons
# - Functional status scores
# - Any Wilcoxon-Mann-Whitney or proportional odds analysis
#
# IF YOU USE THIS CALCULATOR PLEASE CITE:
# Heston, T. F. (2025). Fragility Metrics Toolkit [Software]. Zenodo. https://doi.org/10.5281/zenodo.17254763
#
# © Thomas F. Heston 2025. CC-BY 4.0

import numpy as np
from scipy.stats import norm

ALPHA = 0.05
Z_CRIT = 1.96  # Two-sided critical value at α = 0.05

# ==================== Core calculation ====================
def calculate_ordinal_metrics(gOR, ci_lower, ci_upper):
    """
    Calculate p-fr-nb triplet for ordinal outcomes from generalized odds ratio and 95% CI.

    Parameters:
    -----------
    gOR : float
        Generalized odds ratio (common odds ratio from proportional odds model)
    ci_lower : float
        Lower bound of 95% confidence interval for gOR
    ci_upper : float
        Upper bound of 95% confidence interval for gOR

    Returns:
    --------
    dict with keys:
        - p: approximate two-sided p-value
        - OFQ: Ordinal Fragility Quotient (fr)
        - ORQ: Ordinal Robustness Quotient (nb)
        - z_WMW: Wilcoxon-Mann-Whitney z-statistic
        - ln_gOR: natural log of gOR
        - SE_log_gOR: standard error of ln(gOR)
    """

    # Validation
    if gOR <= 0:
        raise ValueError("gOR must be positive")
    if ci_lower <= 0 or ci_upper <= 0:
        raise ValueError("CI bounds must be positive")
    if ci_lower >= ci_upper:
        raise ValueError("CI lower bound must be less than upper bound")
    if not (ci_lower <= gOR <= ci_upper):
        raise ValueError("gOR must be within the 95% CI bounds")

    # Step 1: Calculate ln(gOR) and SE
    ln_gOR = np.log(gOR)
    ln_ci_lower = np.log(ci_lower)
    ln_ci_upper = np.log(ci_upper)

    # SE from CI width (CI = estimate ± 1.96 × SE)
    SE_log_gOR = (ln_ci_upper - ln_ci_lower) / (2 * Z_CRIT)

    # Step 2: Calculate z-statistic (Wilcoxon-Mann-Whitney approximation)
    z_WMW = ln_gOR / SE_log_gOR

    # Step 3: Calculate OFQ (fragility)
    # Distance from z to critical value (1.96 for two-sided α = 0.05)
    distance_to_boundary = abs(abs(z_WMW) - Z_CRIT)
    OFQ = distance_to_boundary / (1 + distance_to_boundary)

    # Step 4: Calculate ORQ (robustness)
    # Distance from neutrality (gOR = 1 → ln(gOR) = 0)
    ORQ = abs(ln_gOR) / (1 + abs(ln_gOR))

    # Step 5: Approximate p-value (two-sided)
    p_val = 2 * (1 - norm.cdf(abs(z_WMW)))

    return {
        "p": p_val,
        "OFQ": OFQ,
        "ORQ": ORQ,
        "z_WMW": z_WMW,
        "ln_gOR": ln_gOR,
        "SE_log_gOR": SE_log_gOR,
        "gOR": gOR,
        "ci_lower": ci_lower,
        "ci_upper": ci_upper
    }

# ==================== Output formatter ====================
def print_results(results):
    """Pretty-print p-fr-nb results for ordinal outcomes"""
    print("\n" + "="*70)
    print("ORDINAL OUTCOMES CALCULATOR – FRAGILITY_METRICS v10.1")
    print("="*70)
    print(f"Generalized Odds Ratio (gOR) = {results['gOR']:.3f}")
    print(f"95% CI = [{results['ci_lower']:.3f}, {results['ci_upper']:.3f}]")
    print(f"ln(gOR) = {results['ln_gOR']:.4f}")
    print(f"SE[ln(gOR)] = {results['SE_log_gOR']:.4f}")
    print(f"z_WMW = {results['z_WMW']:.3f}")
    print()
    print("-" * 70)
    print("COMPLETE p-fr-nb TRIPLET:")
    print("-" * 70)
    print(f"p-value = {results['p']:.6f}  →  {'SIGNIFICANT' if results['p']<=ALPHA else 'NOT SIGNIFICANT'} at α = 0.05")
    print(f"fr: OFQ = {results['OFQ']:.6f}  (fragility: stability of classification)")
    print(f"nb: ORQ = {results['ORQ']:.6f}  (robustness: distance from neutrality)")
    print()

    # Interpretation guidance
    print("-" * 70)
    print("INTERPRETATION: (subjective - not yet empirically validated)")
    print("-" * 70)

    # Fragility interpretation
    if results['OFQ'] < 0.10:
        fr_interp = "VERY FRAGILE (unstable)"
    elif results['OFQ'] < 0.25:
        fr_interp = "FRAGILE to MILDLY STABLE"
    elif results['OFQ'] < 0.40:
        fr_interp = "MODERATELY STABLE"
    else:
        fr_interp = "VERY STABLE"
    print(f"Fragility: {fr_interp}")

    # Robustness interpretation
    if results['ORQ'] < 0.05:
        nb_interp = "AT NEUTRALITY BOUNDARY"
    elif results['ORQ'] < 0.10:
        nb_interp = "NEAR NEUTRALITY"
    elif results['ORQ'] < 0.25:
        nb_interp = "MODERATE DISTANCE"
    elif results['ORQ'] < 0.50:
        nb_interp = "CLEAR SEPARATION"
    else:
        nb_interp = "FAR FROM NEUTRALITY"
    print(f"Robustness: {nb_interp}")

    # Overall evidence tier (for claims of "effect exists")
    if results['p'] <= ALPHA:
        if results['OFQ'] >= 0.40 and results['ORQ'] >= 0.50:
            tier = "★★★★★ GOLD-STANDARD (highly convincing)"
        elif results['OFQ'] >= 0.25 and results['ORQ'] >= 0.25:
            tier = "★★★★ STRONG"
        elif results['OFQ'] >= 0.10 and results['ORQ'] >= 0.10:
            tier = "★★★ MODERATE"
        else:
            tier = "★★ WEAK/DISCORDANT (significant but fragile and/or near neutrality)"
    else:
        if results['OFQ'] >= 0.25 and results['ORQ'] <= 0.10:
            tier = "★ CREDIBLE NEGATIVE (stable null, near neutrality)"
        else:
            tier = "NOT SIGNIFICANT (interpret based on context)"

    print(f"\nEvidence tier (claiming benefit): {tier}")
    print("="*70 + "\n")

# ==================== Example usage ====================
if __name__ == "__main__":
#    print("\n" + "="*70)
#    print("Ordinal Outcomes Calculator – FRAGILITY_METRICS v10.1")
#    print("For mRS, NIHSS, pain scales, and all ordinal shift analyses")
#    print("="*70 + "\n")
#
#    # Example 1: ANGEL-REBOOT primary outcome (1-year mRS)
#    print("EXAMPLE 1: ANGEL-REBOOT trial (1-year mRS ordinal shift)")
#    print("Reference: Wang et al. JAMA Neurol 2024")
#    print("Treatment: Endovascular therapy for large ischemic stroke")
#    res1 = calculate_ordinal_metrics(gOR=1.34, ci_lower=1.05, ci_upper=1.73)
#    print_results(res1)
#
#    # Example 2: Hypothetical strong effect
#    print("\n" + "="*70)
#    print("EXAMPLE 2: Hypothetical strong ordinal shift")
#    res2 = calculate_ordinal_metrics(gOR=2.5, ci_lower=1.8, ci_upper=3.5)
#    print_results(res2)
#
#    # Example 3: Fragile result
#    print("\n" + "="*70)
#    print("EXAMPLE 3: Hypothetical fragile result (barely significant)")
#    res3 = calculate_ordinal_metrics(gOR=1.18, ci_lower=1.01, ci_upper=1.38)
#    print_results(res3)
#
#    # Interactive mode
#    print("\n" + "="*70)
#    print("INTERACTIVE MODE")
#    print("="*70)
#
#    try:
#  use_interactive = input("\nRun interactive mode? (y/n): ").lower()

#        if use_interactive == 'y':
  print("\nEnter ordinal outcome data:")
  print("(Typically reported as: 'adjusted common OR = X.XX, 95% CI X.XX-X.XX')")
  gOR = float(input("\nGeneralized odds ratio (gOR): "))
  ci_lower = float(input("95% CI lower bound: "))
  ci_upper = float(input("95% CI upper bound: "))

  results = calculate_ordinal_metrics(gOR, ci_lower, ci_upper)
  print_results(results)

#    except KeyboardInterrupt:
#        print("\n\nExiting...")
#    except Exception as e:
#        print(f"\nError: {e}")
#        print("Please check your inputs and try again.")

print("\n" + "="*70)
#print("Citation:")
print("Heston, T. F. (2025). Fragility Metrics Toolkit [Software].")
print("Zenodo. https://doi.org/10.5281/zenodo.17254763")
print("="*70 + "\n")